In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc))
# print(inspect.getsource(apps.backward_geocode_slc))

# scene

In [ ]:
dem_urlpath = "data/Chicago-4m-DEM.tif"

dem_raster = scene.open_dem_raster(dem_urlpath, chunks=2500)
dem_raster

In [ ]:
_ = dem_raster.plot()

In [ ]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

In [ ]:
%%time
dem_normal_ecef = scene.compute_diff_normal(dem_ecef)
dem_normal_ecef

# image

In [ ]:
product_urlpath = (
    "data/S1B_S6_SLC__1SDV_20211216T115438_20211216T115501_030050_03968A_4DCB.SAFE"
)
measurement_group = "S6/VV"
orbit_group = f"{measurement_group}/orbit"
calibration_group = f"{measurement_group}/calibration"
output_urlpath = "Chicago-4m-GTC-SLC.tif"

!ls -d {product_urlpath}

In [ ]:
measurement = xr.open_dataarray(
    product_urlpath,
    engine="sentinel-1",
    group=measurement_group,
    chunks=2048,
)
measurement

In [ ]:
calibration = xr.open_dataset(
    product_urlpath, engine="sentinel-1", group=calibration_group
)
beta_nought_lut = calibration.betaNought
beta_nought_lut

In [ ]:
%%time
beta_nought = xarray_sentinel.calibrate_amplitude(abs(measurement), beta_nought_lut)
beta_nought.attrs.update(measurement.attrs)
beta_nought

In [ ]:
orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group)
position_ecef = orbit_ecef.position
position_ecef

In [ ]:
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(position_ecef)
position_ecef = orbit_interpolator.position()
velocity_ecef = orbit_interpolator.velocity()
position_ecef

In [ ]:
%%time
dem_coords = geocoding.backward_geocode(dem_ecef, position_ecef, velocity_ecef)
dem_coords

In [ ]:
%%time
geocoded = beta_nought.interp(
        azimuth_time=dem_coords.azimuth_time,
        slant_range_time=dem_coords.slant_range_time,
        method="nearest",
    )

geocoded

In [ ]:
_ = geocoded.plot(vmax=1.0)

In [ ]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
cos_incidence_angle = xr.dot(
    dem_normal_ecef,
    -dem_coords["dem_direction"],
    dims="axis",
)
sin_incidence_angle = np.sin(np.arccos(cos_incidence_angle))
sin_incidence_angle

In [ ]:
(1 / sin_incidence_angle).plot(vmax=3)

In [ ]:
rtc = geocoded * sin_incidence_angle

In [ ]:
_ = rtc.plot(vmax=1.0)

In [ ]:
rtc.rio.set_crs(dem_raster.rio.crs)
rtc.rio.to_raster(
    output_urlpath.replace("GTC", "RTC"),
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)